In [2]:
import pickle
from flask import Flask, request, jsonify
import pandas as pd

In [3]:
RF = pickle.load(open('../Pickle/RandomForest.pkl', 'rb'))

app = Flask(__name__)

def process_data(data):

    df = pd.DataFrame(data, index=[0])

    with open('encoder.pkl', 'rb') as f:
        encoder = pickle.load(f)

    cols = ["city", "state", "zip_code", "status", "sold"]
    encoded_user_data = encoder.transform(df[cols])

    encoded_user_data['bed'] = df['bed']
    encoded_user_data['bath'] = df['bath']
    encoded_user_data['acre_lot'] = df['acre_lot']
    encoded_user_data['house_size'] = df['house_size']
    encoded_user_data.drop(columns=['zip_code_A>'], inplace=True)

    return encoded_user_data


@app.route('/predict', methods=['POST'])
def predict():
    data = request.get_json()
    data = process_data(data)
    df = pd.DataFrame.from_dict(data)
    prediction = RF.predict(df)
    return jsonify(prediction.tolist())

@app.route('/retrain', methods=['POST'])
def retrain():
    # read data from database
    # assign X_train adn y_train
    # RF.fit(X_train, y_train)

    # overwrite the pickle to save the new training
    with open('RandomForest.pkl', 'wb') as model_pkl:
        pickle.dump(RF, model_pkl)

try:
    if __name__ == '__main__':
        app.run(port=5000, debug= False)
except Exception as e:
    print('Error:', e)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
